## RL agent Q-learning for TicTacToe env

The [Tic-Tac-Toe](https://github.com/MauroLuzzatto/OpenAI-Gym-TicTacToe-Environment) is a simple game environment that allows to train reinforcement learning agents.

In [1]:
# load the python modules
import time
import sys
import warnings

import gym
import numpy as np
from tqdm import tqdm
import gym_TicTacToe

from src.qagent import QLearningAgent
from src.play_tictactoe import play_tictactoe, play_tictactoe_with_random

from src.utils import (
    create_state_dictionary,
    reshape_state,
    save_qtable,
    load_qtable
)

# ignore warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [2]:
class Player:
    def __init__(self, color, episodes: int):
        self.color = color
        self.reward_array = np.zeros(episodes)
        self.reset_reward()
        self.name = f"Player {color}"

    def reset_reward(self):
        self.reward = 0

In [3]:
# initialize the tictactoe environment
env = gym.envs.make("TTT-v0", small=-1, large=10)

In [4]:
state_dict = create_state_dictionary()
state_size = len(state_dict.keys())
action_size = env.action_space.n

Number of legal states: 17906


In [5]:
# set training parameters
episodes = 260_000
max_steps = 9

In [6]:
exploration_parameters = {
    "max_epsilon": 1.0,
    "min_epsilon": 0.0,
    "decay_rate": 0.00001,
}

In [7]:
qagent = QLearningAgent(exploration_parameters, state_size, action_size, learning_rate= 0.55, gamma=0.99)

In [8]:

def play(qagent:QLearningAgent, player: Player, state: int, action_space: np.array) -> tuple:
    action = qagent.get_action(state, action_space)

    # remove action from the action space
    action_space = action_space[action_space != action]

    new_state, reward, done, _ = env.step((action, player.color))
    #TODO: maybe should change a marker after this agent turn 
    new_state = np.append(new_state, player.color)
    new_state = state_dict[reshape_state(new_state)]    

    qagent.qtable[state, action] = qagent.update_qtable(
        state, new_state, action, reward, done
    )
    # new state
    state = new_state
    return state, action_space, done

In [9]:
def play_random(qagent:QLearningAgent, player: Player, state: int, action_space: np.array) -> tuple:
    action = np.random.choice(action_space)
    action_space = action_space[action_space != action]
    new_state, reward, done, _ = env.step((action, player.color))
    new_state = np.append(new_state, player.color)
    new_state = state_dict[reshape_state(new_state)]
    state = new_state
    return state, action_space, done

In [10]:
visited_states = np.zeros((state_size, 1))

In [11]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
import random

start_time = time.time()

player_1 = Player(color=1, episodes=episodes)
player_2 = Player(color=2, episodes=episodes)

track_progress = np.zeros(episodes)

win_history = []

rewards = []
for episode in tqdm(range(episodes)):
    
    action_space = np.arange(9)

    player_1.reset_reward()
    player_2.reset_reward()

    # randomly change the order players
    start = np.random.choice([1,2])

    state, _ = env.reset()
    state = np.append(state, start)
    state = state_dict[reshape_state(state)]

    for _step in range(start, max_steps + start):

        # change a turn
        if _step % 2 == 0:
            #state, action_space, done = play_random(qagent, player_1, state, action_space)
            state, action_space, done = play(qagent, player_1, state, action_space)
        else:
            state, action_space, done = play(qagent, player_2, state, action_space)
        visited_states[state] += 1
        if done == True:
            break

    # reduce epsilon for exporation-exploitation tradeoff
    qagent.update_epsilon(episode)

    #check how good is agent
    if episode % 1_000 == 0:
        num_games = 50
        cur_win_rate, reward = play_tictactoe_with_random(env, qagent.qtable, state_dict, num_test_games=num_games)
        win_history.append(sum(cur_win_rate)/num_games)
        print("WinRate:", sum(cur_win_rate)/num_games)
        # rewards.append(reward)
        # clear_output(True)
        # # plt.title('eps = {:e}, mean reward = {:.1f}'.format(agent.epsilon, np.mean(rewards[-10:])))
        # plt.plot(rewards)
        # plt.show()
    if episode % 1_0000 == 0:

        sum_q_table = np.sum(qagent.qtable)
        time_passed = round((time.time() - start_time) / 60.0, 2)

        print(
            f"episode: {episode}, \
            epsilon: {round(qagent.epsilon, 2)}, \
            sum q-table: {sum_q_table}, \
            elapsed time [min]: {time_passed},  \
            done [%]: {episode / episodes * 100} \
            "
        )


  0%|          | 0/260000 [00:00<?, ?it/s]

AttributeError: 'Player' object has no attribute 'add_reward'

In [55]:
num_games = 1000
cur_win_rate, _ = play_tictactoe_with_random(env, qagent.qtable, state_dict, num_test_games=num_games)
win_history.append(sum(cur_win_rate)/num_games)
print("WinRate:", sum(cur_win_rate)/num_games)

WinRate: 0.678


In [43]:
qtable = qagent.qtable
save_qtable(qtable, 'tables', "q_table_070")

q_table_070.npy saved!


In [ ]:
q_table = load_qtable('tables', "q_table_067")

In [59]:
play_tictactoe(env, qagent.qtable, state_dict, num_test_games=1)

Agent beginns
--------------------
╒═══╤═══╤═══╕
│ - │ - │ - │
├───┼───┼───┤
│ - │ - │ - │
├───┼───┼───┤
│ - │ - │ - │
╘═══╧═══╧═══╛
--------------------
Move Human
Action: 4
-1


╒═══╤═══╤═══╕
│ - │ - │ - │
├───┼───┼───┤
│ - │ X │ - │
├───┼───┼───┤
│ - │ - │ - │
╘═══╧═══╧═══╛
--------------------
move Agent
Action: 3


╒═══╤═══╤═══╕
│ - │ - │ - │
├───┼───┼───┤
│ O │ X │ - │
├───┼───┼───┤
│ - │ - │ - │
╘═══╧═══╧═══╛
--------------------
Move Human
Action: 0
-1


╒═══╤═══╤═══╕
│ X │ - │ - │
├───┼───┼───┤
│ O │ X │ - │
├───┼───┼───┤
│ - │ - │ - │
╘═══╧═══╧═══╛
--------------------
move Agent
Action: 8


╒═══╤═══╤═══╕
│ X │ - │ - │
├───┼───┼───┤
│ O │ X │ - │
├───┼───┼───┤
│ - │ - │ O │
╘═══╧═══╧═══╛
--------------------
Move Human
Action: 1
-1


╒═══╤═══╤═══╕
│ X │ X │ - │
├───┼───┼───┤
│ O │ X │ - │
├───┼───┼───┤
│ - │ - │ O │
╘═══╧═══╧═══╛
--------------------
move Agent
Action: 7


╒═══╤═══╤═══╕
│ X │ X │ - │
├───┼───┼───┤
│ O │ X │ - │
├───┼───┼───┤
│ - │ O │ O │
╘═══╧═══╧═══╛
------

In [78]:
# state = np.random.choice(np.arange(env.observation_space.n))
# # state_dict[state]
# print(state)

# key = list(filter(lambda x: state_dict[x] == state, state_dict))[0]
# print(np.array(key[:-1]).reshape(3,3))
# print("Turn was:", key[-1])
# print(np.round(qagent.qtable[state].reshape(3,3),1))

4919
[[1 2 0]
 [0 2 1]
 [0 0 2]]
Turn was: 2
[[0.  0.  7.2]
 [7.2 0.  0. ]
 [7.7 5.8 0. ]]
